In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import scipy.sparse
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import silhouette_score


from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data_path = '../data/RETINA/'

In [ ]:
# load clusters from author
clusters = pd.read_csv(data_path + "ClustAssignFile.txt", index_col=0, sep="\t")[["CLUSTER"]]
clusters = clusters[clusters.CLUSTER <= 15]

# load batch info from online
batch_id = pd.read_csv(data_path + "batch_id.csv", index_col=0)
encoder = OneHotEncoder()
batch = encoder.fit_transform(batch_id)
batch_id = batch_id.values[:, 0]

# load barcodes, combat and expression data from online
barcodes = [x[0] for x in pd.read_csv(data_path + "barcodes.csv", index_col=0).values.astype(np.str)]
expression = scipy.io.mmread(data_path + "count.mtx").T.A
pca_combat = pd.read_csv(data_path + "pca_combat.csv", index_col=0)

# slice to only cells from identified cell_types
expression = pd.DataFrame(data = expression, index=barcodes).loc[clusters.index]
batch_id = pd.DataFrame(data = batch_id, index=barcodes).loc[clusters.index]
pca_combat = pd.DataFrame(data = pca_combat, index=barcodes).loc[clusters.index]
clusters = np.array([x[0] for x in clusters.values])
print(expression.shape , pca_combat.shape, batch.shape, clusters.shape)

In [ ]:
np.unique(clusters)

In [ ]:
expression_train, expression_test, combat_train, combat_test, b_train, b_test, c_train, c_test = \
                train_test_split(expression.as_matrix(), pca_combat.as_matrix(), batch_id.as_matrix(), \
                                 clusters, random_state=0)

In [ ]:
scipy.sparse.save_npz(data_path + "data_train.npz", scipy.sparse.coo_matrix(expression_train))
scipy.sparse.save_npz(data_path + "data_test.npz", scipy.sparse.coo_matrix(expression_test))

np.savetxt(data_path + "batch_train", b_train)
np.savetxt(data_path + "batch_test", b_test)

np.savetxt(data_path + "combat_train", combat_train)
np.savetxt(data_path + "combat_test", combat_test)

np.savetxt(data_path + "c_train", c_train)
np.savetxt(data_path + "c_test", c_test)